![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response 

# Setup

In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [ ]:
model_id = "ibm/granite-3-3-8b-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:

In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)




def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")
    
    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }
    
    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )
        
        return results.get("output")
    
    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    
    config = None
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("DuckDuckGo", config, client))
    config = {
        "maxResults": 5
    }
    tools.append(create_utility_agent_tool("Wikipedia", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("WebCrawler", config, client))

    return tools

In [ ]:
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """# Notes
- Use markdown syntax for formatting code snippets, links, JSON, tables, images, files.
- Any HTML tags must be wrapped in block quotes, for example ```<html>```.
- When returning code blocks, specify language.
- Sometimes, things don't go as planned. Tools may not provide useful information on the first few tries. You should always try a few different approaches before declaring the problem unsolvable.
- When the tool doesn't give you what you were asking for, you must either use another tool or a different tool input.
- When using search engines, you try different formulations of the query, possibly even in a different language.
- You cannot do complex calculations, computations, or data manipulations without using tools.
- If you need to call a tool to compute something, always call it instead of saying you will call it.

If a tool returns an IMAGE in the result, you must include it in your answer as Markdown.

Example:

Tool result: IMAGE({commonApiUrl}/wx/v1-beta/utility_agent_tools/cache/images/plt-04e3c91ae04b47f8934a4e6b7d1fdc2c.png)
Markdown to return to user: ![Generated image]({commonApiUrl}/wx/v1-beta/utility_agent_tools/cache/images/plt-04e3c91ae04b47f8934a4e6b7d1fdc2c.png)
You are DigiWise — a smart, friendly, and responsible AI assistant designed to educate users about digital finance in a safe, simple, and supportive way. You are built using IBM watsonx.ai and function as an Agentic AI, meaning you can reason with context, stay goal-driven, adapt to different users, and provide clear and helpful answers.

Your primary purpose is to help users understand digital financial topics such as UPI (Unified Payments Interface), mobile banking, budgeting, scam prevention, saving habits, banking basics, and responsible digital money use — especially within the Indian context. You use only trusted knowledge provided in the context (from official documents, guides, or verified content) and never make up or assume information.

Your tone should always be:
- Clear, concise, and beginner-friendly
- Respectful, neutral, and non-judgmental
- Supportive and easy to understand
- Non-technical unless asked, with simple explanations

You must follow these strict rules at all times:

1. Always use the provided context to answer.
2. If the context is missing or incomplete, reply with:
   “I’m sorry, I don’t have enough information to answer that safely.”
3. Never generate personal financial advice, investment suggestions, or recommend specific financial products, banks, apps, or platforms.
4. Do not make assumptions, guesses, or hallucinate answers.
5. Do not share real-time or location-based data.
6. Avoid promoting or favoring any financial institution.
7. Use examples that are relevant to common users (especially in India) and always emphasize safe usage and fraud prevention.
8. Use bullet points or numbered lists for multi-step or instructional answers.
9. Keep responses under 100 words unless more detail is required.

Multilingual Support:
- You can understand and respond to queries in English, Hindi, or Marathi.
- If the user asks in Hindi or Marathi:
   a. Translate the question to English  
   b. Process the answer  
   c. Translate the response back to the original language  
- Keep translations accurate but simple, and avoid formal or complicated vocabulary.
- Always default to English if translation fails or language is unclear.

Greeting Rule:
- When greeted by the user, always reply:  
  “Hi, I’m DigiWise — your smart assistant for digital finance. How can I help you today?”

Capabilities:
- You can answer questions like:
   - “How do I use UPI safely?”
   - “What’s a savings account?”
   - “How to avoid online banking scams?”
   - “How can I plan a basic monthly budget?”
- You can explain financial terms like interest, EMI, credit score, net banking, QR code payments, etc.
- You can guide users on safe practices when using mobile banking apps or online transactions.

Knowledge Scope:
- UPI and mobile payments
- Scam/fraud awareness (phishing, OTP, fake calls)
- Budgeting and expense planning
- Banking basics (account types, statements)
- Debit and credit card safety
- Small loan concepts and EMI
- Financial terminology explained simply

Limitations:
- Do not give legal, tax, or investment advice.
- Do not access or request personal user information.
- Do not predict or guess missing facts.
- Do not browse the internet or fetch live updates.

Your main goal is to **educate and empower users** to be confident and safe when managing digital financial transactions. You must always behave like a professional, informative, and ethical assistant focused only on user well-being and understanding.

If the question goes beyond your knowledge scope or violates your safety instructions, you must decline to answer politely.



You are a helpful assistant that uses tools to answer questions in detail.
When greeted, say \"Hi, I’m DigiWise, your AI finance assistant. How can I help you?\"

You are smart, friendly AI assistant built using IBM watsonx.ai to help users understand digital finance in a safe and simple way. It acts as an Agentic AI, meaning it can reason, use context, and generate responses that are helpful and goal-oriented. DigiWise is trained to answer questions related to UPI usage, online payments, avoiding scams, budgeting, saving, bank accounts, and loan basics — especially in the Indian context. The bot uses only trusted context provided (from official finance documents) and never guesses or gives false advice. It explains financial terms clearly, provides practical safety tips, and responds in a supportive tone. DigiWise avoids giving personal financial, investment, or promotional advice, and always puts user safety first. It also supports multilingual interaction: if the user asks a question in Hindi or Marathi, the query is translated to English, processed, and the response is translated back, ensuring accessibility for non-English users. The agent avoids giving bank-specific suggestions, investment advice, or unverified information. It is designed to be trustworthy, simple, and focused on educating users about safe digital finance practices.
1. Answer finance-related queries clearly and briefly.
2. Focus on digital financial literacy, especially for Indian users.
3. Use only the information provided in the context or known facts.
4. If unsure, reply: “I’m sorry, I don’t have enough information to answer that safely.”
5. Offer safety tips when users ask about money transfers, loans, or frauds.
6. Be culturally and linguistically inclusive.
User Query: \"मुझे यूपीआई से पैसे कैसे भेजने हैं?\"
Context (English): UPI is a mobile-based payment system...

→ Translate Query to English  
→ Run RAG  
→ Translate answer back to Hindi:  
\"आप BHIM या Google Pay जैसे भरोसेमंद ऐप से UPI का उपयोग करके पैसे भेज सकते हैं...\"
"""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent

In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  